# Arize Setup

In [1]:
# !pip install -qe '.'
!pip install -qe '.[ml-stream]'
# !pip install -q 'arize[spans]==8.0.0a2'
!pip freeze | grep arize

-e git+ssh://git@github.com/Arize-ai/arize.git@853d50dd2866f19d0600e4eb740c9e9aa28bd27f#egg=arize&subdirectory=sdk/python/loggerv8


In [2]:
SPACE_ID = "U3BhY2U6NTA3MDpsTlIr"
MODEL_NAME = "test-sdkv8-stream-09-30-25-c"

In [3]:
import os

os.environ["ARIZE_LOG_ENABLE"] = "true"
os.environ["ARIZE_LOG_LEVEL"] = "debug"
os.environ["ARIZE_LOG_STRUCTURED"] = "false"

# Stream ML record 

In [4]:
import logging

from arize import ArizeClient

client = ArizeClient() # API key from env var

print("arize handlers:", logging.getLogger("arize").handlers)
print(
    "arize.spans.client handlers:",
    logging.getLogger("arize.spans.client").handlers,
)

print(client.sdk_config)

arize handlers: [<StreamHandler stdout (DEBUG)>]
arize.spans.client handlers: []
SDKConfiguration(
  api_key='ak-ed5***',
  api_host='api.arize.com',
  api_scheme='https',
  flight_server_host='flight.arize.com',
  flight_server_port=443,
  flight_scheme='grpc+tls',
  request_verify=True,
  stream_max_workers=8,
  stream_max_queue_bound=5000,
)


In [5]:
import numpy as np

from arize.types import ArizeTypes, Embedding, TypedValue

# Example features; features & tags can be optionally defined with typing
features = {
    "state": "ca",
    "city": "berkeley",
    "merchant_name": "Peets Coffee",
    "pos_approved": TypedValue(value=False, type=ArizeTypes.INT),
    "item_count": 10,
    "merchant_type": "coffee shop",
    "charge_amount": TypedValue(value=20.11, type=ArizeTypes.FLOAT),
}

# Example embedding features
embedding_features = {
    "nlp_embedding": Embedding(
        vector=np.array([4.0, 5.0, 6.0, 7.0]),
        data="This is a test sentence",
    ),
}

In [6]:
from arize.types import Environments, ModelTypes

response = client.models.log_stream(
    space_id=SPACE_ID,
    model_name=MODEL_NAME,
    model_type=ModelTypes.SCORE_CATEGORICAL,
    environment=Environments.PRODUCTION,
    prediction_label=("not fraud", 0.3),
    actual_label=("fraud", 1.0),
    features=features,
    embedding_features=embedding_features,
)

# If successful, the server will return a status_code of 200
print(response.result())

<Response [200]>


In [7]:
response.result()

<Response [200]>